In [1]:
#导入依赖库
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader,TensorDataset
from sklearn.preprocessing import StandardScaler

In [2]:
#参考文献doi:10.1039/c9ta12608b

#读取数据表格
df = pd.read_csv('./NRR数据集完整版.csv')
print(df)
#取出输入描述符X(倒数第7到倒数第2列)和标签Y(最后1列Eligible-作为NRR催化剂的可行性：1-可行，0-不可行，二分类)
#各特征含义见文献
#为简化演示，删去了库伦矩阵的PCA，并且仅以N2吸附能作为可行性判据
data = df.iloc[:,-7:]
print(data)
#转化为numpy数组
data = data.to_numpy()
#特征数据标准化(可能提高模型性能，注释该部分代码以跳过)
data[:,-7:-1] = StandardScaler().fit_transform(data[:,-7:-1])
#shuffle数据集
np.random.shuffle(data)

#分配训练集和验证集
#总样本数和训练样本数
num_sample=data.shape[0]
num_train=int(num_sample*0.8)
print(num_train)
#描述符和标签切片
train_data=data[:num_train,:-1]
train_lab=data[:num_train,-1]
test_data=data[num_train:,:-1]
test_lab=data[num_train:,-1]

      M Structure Adsorption species orientation       DFT      LGBM  \
0     V       @B4                *N2    (end-on) -1.000218 -1.015938   
1     V       @B4                *N2   (side-on) -0.821077 -0.855509   
2    Cu       @B4                *N2    (end-on) -0.497864 -0.465921   
3    Cu       @B4                *N2   (side-on) -0.198403 -0.273015   
4    Tc       @B4                *N2    (end-on) -1.341411 -1.238400   
..   ..       ...                ...         ...       ...       ...   
123  Tc       @B3                *N2    (end-on) -0.720426 -0.985053   
124  Tc       @B3                *N2   (side-on) -0.345986 -0.499503   
125  Ag       @B3                *N2   (side-on) -0.196123 -0.449861   
126  Ir       @B3                *N2    (end-on) -0.700367 -0.696542   
127  Ir       @B3                *N2   (side-on) -0.248328 -0.343387   

    Type of data  NM   fB  NC     X     Z      R  Eligible  
0       Training   1  1.0   4  1.63  23.0  252.0       1.0  
1       Train

In [24]:
#构建文献中2个10维隐藏层的分类MLP
#获取运行环境
# device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')
print(f"Using {device} device")

#定义模型(复习)
class NN(nn.Module):
    def __init__(self, input_dim):
        super(NN, self).__init__()
        #利用Sequential容器快速定义一个2层10维的MLP模块
        self.mlp = nn.Sequential(
            nn.Linear(input_dim, 10),
            nn.ReLU(),
            nn.Linear(10, 2),
        )

    def forward(self, x):
        out = self.mlp(x)
        #二分类可使用logistic回归
        out = torch.sigmoid(out)
        return out

model = NN(input_dim=train_data.shape[1]).to(device)
print(model)

Using cpu device
NN(
  (mlp): Sequential(
    (0): Linear(in_features=6, out_features=10, bias=True)
    (1): ReLU()
    (2): Linear(in_features=10, out_features=2, bias=True)
  )
)


In [25]:
#定义测试集上准确率(Accuracy)的计算方法
def test(pred,lab):
    t=pred.max(-1)[1]==lab
    return torch.mean(t.float())

In [26]:
#其他超参
criterion=nn.CrossEntropyLoss() # 使用CrossEntropyLoss损失
optm=torch.optim.Adam(model.parameters(),lr=0.001) # 优化器
epochs=500 # 训练500次

In [27]:
for i in range(epochs):
    # 指定模型为训练模式，计算梯度
    model.train()
    # 输入值都需要转化成torch的Tensor
    x=torch.from_numpy(train_data).float()
    y=torch.from_numpy(train_lab).long()
    y_hat=model(x)
    loss=criterion(y_hat,y) # 计算损失
    optm.zero_grad() # 前一步的损失清零
    loss.backward() # 反向传播
    optm.step() # 优化
    if (i+1)%50 ==0 : # 这里我们每10次输出相关的信息
        # 指定模型为评估模式
        model.eval()
        test_in=torch.from_numpy(test_data).float()
        test_l=torch.from_numpy(test_lab).long()
        test_out=model(test_in)
        # 使用我们的测试函数计算准确率
        accu=test(test_out,test_l)
        print("Epoch:{},Loss:{:.4f},Accuracy：{:.2f}".format(i+1,loss.item(),accu))

Epoch:50,Loss:0.6854,Accuracy：0.58
Epoch:100,Loss:0.6529,Accuracy：0.69
Epoch:150,Loss:0.6105,Accuracy：0.65
Epoch:200,Loss:0.5668,Accuracy：0.69
Epoch:250,Loss:0.5316,Accuracy：0.69
Epoch:300,Loss:0.5075,Accuracy：0.73
Epoch:350,Loss:0.4902,Accuracy：0.73
Epoch:400,Loss:0.4766,Accuracy：0.73
Epoch:450,Loss:0.4660,Accuracy：0.73
Epoch:500,Loss:0.4578,Accuracy：0.73


In [16]:
#可以使用一下这个模型
model.eval()
#预测前20条数据
test_df=df[:20]
#输出中心金属单原子和配位结构，相应的标签，用于和预测结果对比
print(test_df.iloc[:20,[0,1,-1]])
#取出输入描述符X
x = test_df.iloc[:,-7:-1]
x = x.to_numpy()
x = StandardScaler().fit_transform(x)
x = torch.from_numpy(x).float()
#预测
y_hat = model(x)
y_hat = y_hat.max(-1)[-1]
print(y_hat)

     M Structure  Eligible
0    V       @B4       1.0
1    V       @B4       1.0
2   Cu       @B4       0.0
3   Cu       @B4       0.0
4   Tc       @B4       1.0
5   Tc       @B4       1.0
6   Ru       @B4       1.0
7   Ru       @B4       1.0
8   Rh       @B4       0.0
9   Rh       @B4       0.0
10  Mo       @B4       1.0
11  Mo       @B4       1.0
12  Au       @B4       0.0
13  Au       @B4       0.0
14  Ag       @B4       0.0
15  Ag       @B4       0.0
16  Ir       @B4       1.0
17  Ir       @B4       0.0
18  Pt       @B4       0.0
19  Pt       @B4       0.0
tensor([1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0])
